In [1]:
using Pkg
Pkg.activate("../..")
using ClimaLSM
using ClimaLSM.DALEC
using CSV
using DataFrames
using NetCDF
using OrdinaryDiffEq: solve, DiscreteProblem, FunctionMap
using Plots
Plots.pyplot();

  Activating project at `~/.julia/dev/ClimaLSM.jl`


In [2]:
using NetCDF
"""
    prepare_drivers(CBF_PATH::String, FT::Type{T}) where {T <: AbstractFloat}

Prepare the drivers from DALEC811 model.
    - `CBF_PATH::String`: a String to the CBF file 
    - `FT::Type{T}`: Float type, should be a subtype of AbstractFloat
"""
function prepare_drivers(CBF_PATH::String, FT::Type{T}) where {T <: AbstractFloat}
    
    TIME_data = convert.(FT, ncread(CBF_PATH, "time"))
    TIME(t::Integer) = TIME_data[t]
    
    NODAYS = length(TIME_data)

    T_MIN_data = convert.(FT, ncread(CBF_PATH, "T2M_MIN"))
    T_MIN(t::Integer) = T_MIN_data[t]

    T_MAX_data = convert.(FT, ncread(CBF_PATH, "T2M_MAX"))
    T_MAX(t::Integer) = T_MAX_data[t] 

    RADIATION_data = convert.(FT, ncread(CBF_PATH, "SSRD"))
    SSRD(t::Integer) = RADIATION_data[t]

    ATMOSPHERIC_CO2_data = convert.(FT, ncread(CBF_PATH, "CO2"))
    ATMOSPHERIC_CO2(t::Integer) = ATMOSPHERIC_CO2_data[t]

    DOY_data = convert.(FT, ncread(CBF_PATH, "DOY"))
    DOY(t::Integer) = DOY_data[t]

    BURNED_AREA_data = convert.(FT, ncread(CBF_PATH, "BURNED_AREA"))
    BURNED_AREA(t::Integer) = BURNED_AREA_data[t]

    VPD_data = convert.(FT, ncread(CBF_PATH, "VPD"))
    VPD(t::Integer) = VPD_data[t]

    PRECIPITATION_data = convert.(FT, ncread(CBF_PATH, "TOTAL_PREC"))
    PRECIPITATION(t::Integer) = PRECIPITATION_data[t]
    
    LATITUDE = convert.(FT, ncread(CBF_PATH, "LAT"))
    MEAN_TEMP = sum((T_MAX_data .+ T_MIN_data) ./ 2) / length(T_MAX_data)
    MEAN_PRECIP = sum(PRECIPITATION_data) / length(PRECIPITATION_data)
    DELTA_T = FT(TIME_data[2]-TIME_data[1])

    atmos = DALEC811AtmosphericDrivers(T_MIN, T_MAX, ATMOSPHERIC_CO2, VPD, PRECIPITATION, LATITUDE, MEAN_TEMP, MEAN_PRECIP, FT)
    rad = DALEC811RadiativeDrivers(SSRD, FT)
    fire = DALEC811FireDrivers(BURNED_AREA, FT)
    temporal = DALEC811TemporalDrivers(TIME, DOY, DELTA_T, NODAYS, FT)
    
    return atmos, rad, fire, temporal
end

prepare_drivers

In [3]:
FT = Float64
CBF_PATH = "./data/AU-How_example_driver.nc.cbf"
PARAM_PATH = "./data/AU-How_example_param.csv"

"./data/AU-How_example_param.csv"

In [ ]:
atmos, rad, fire, temporal = prepare_drivers(CBF_PATH, FT);
d = ClimaLSM.Domains.Point(; z_sfc=0.0)
dalec_params = FT.(CSV.read(PARAM_PATH, DataFrame).dalec811_params);
check_dalec_811_parameter_bounds(dalec_params)
parameters = DALEC811Parameters(dalec_params...);
photosynthesis = ACM(FT)

model = DALECModel(atmos, rad, fire, temporal, photosynthesis, parameters, d);
Y, p, coord = ClimaLSM.initialize(model);
load_initial_condition!(model, Y);

trange=(0,model.temporal.NODAYS)
ode_function! = ClimaLSM.make_rhs(model)
prob = DiscreteProblem(ode_function!, Y, trange, p)
sol=solve(prob, FunctionMap{true}());

In [ ]:
dalec_vars = prognostic_vars(model)
lai_plot = plot(sol[1, 2:model.temporal.NODAYS+1], ylabel=dalec_vars[1], xlabel="month", label=nothing)
gpp_plot = plot(sol[2, 2:model.temporal.NODAYS+1], ylabel=dalec_vars[2], xlabel="month", label=nothing)
et_plot = plot(sol[3, 2:model.temporal.NODAYS+1], ylabel=dalec_vars[3], xlabel="month", label=nothing)
nee_plot = plot(sol[32, 2:model.temporal.NODAYS+1], ylabel=dalec_vars[32], xlabel="month", label=nothing)
plot(lai_plot, gpp_plot, et_plot, nee_plot)